Cеминар 5. Реализация алгоритма LZ78
-----------------------------------------------

***********************************************

*Материалы:*

[Википедия](https://ru.wikipedia.org/wiki/LZ77)  
[Викиконспекты](http://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_LZ77_%D0%B8_LZ78)  
[Хабр](https://habrahabr.ru/post/132683/)

***********************************************

1. Вспомогательные функции
---------------------------------------------

`find_pattern` - находит подстроку максимальной длины, находящейся в списке  
`int_to_byte`  - переводит целое число в двоичный вид с заданной длиной

*********************************************

In [1]:
def find_pattern(dictionary, text):
    # Просматриваем слова в порядке возрастания
    for element in sorted(dictionary, key=lambda x: len(x), reverse=True):
        # Обрабатываем случай неправильной работы zip
        if len(element) < len(text):
            # Если все пары одинаковы - текст начинался с данного элемента
            if all(a == b for a, b in zip(element, text)):
                return element
            
def int_to_byte(element, base):
    return ('{:0'+ str(base) + 'b}').format(element)

2. Функция кодирования
---------------------------------------------

Параметр `DICT_MAX_LENGTH` показывает максимально возможный размер словаря, при превышении которого он очищается

*********************************************

In [2]:
import time
import math

DICT_MAX_LENGTH = 1024 # 2^n - 1
CODE_BYTE_LENGTH = int(math.ceil(math.log(DICT_MAX_LENGTH, 2))) # Сколько бит занимает 1 элемент кода
ASCII_BYTE_LENGTH = 8
CODE_LENGTH = ASCII_BYTE_LENGTH + CODE_BYTE_LENGTH

def encode_lz78(text):
    
    time_enter = time.time()
    
    dictionary = []
    codes = []
    overflow_count = 0
    
    # Составляем пары    
    while len(text):
        element = find_pattern(dictionary, text)
        if element:
            codes.append((dictionary.index(element) + 1, text[len(element)]))
            dictionary.append(element + text[len(element)])
            text = text[len(element) + 1:]
        else:
            codes.append((0, text[0]))
            dictionary.append(text[0])
            text = text[1:]

        # Очищаем словарь при переполнении
        if len(dictionary) > DICT_MAX_LENGTH:
            dictionary = []
            overflow_count += 1

    print 'Количество переполнений буффера {}'.format(overflow_count)
    print 'Количество пар {}'.format(len(codes))
    
    # Переводим в бинарник
    result = ''    
    for element in codes:
        result += int_to_byte(element[0], CODE_BYTE_LENGTH)
        result += int_to_byte(ord(element[1]), ASCII_BYTE_LENGTH)
        
    print 'Время выполнения {:.3f}'.format(time.time() - time_enter)
    
    return result, len(codes)

3. Генерация фэйкового текста
-----------------------------------------------

In [3]:
from faker import Factory

fake = Factory.create('en_US')

text = fake.text(10000)
print text[:1000] + '...'

Veritatis harum aperiam praesentium temporibus repudiandae. Totam asperiores blanditiis voluptas mollitia. Praesentium aut tenetur voluptates nam ducimus.
In ullam fuga at totam. Eligendi reprehenderit sapiente sapiente cum error ea. Ad dignissimos voluptate inventore cum ratione quae. Necessitatibus iste excepturi itaque pariatur. Repudiandae consectetur maiores temporibus quo autem vero illo.
Nesciunt tempora atque facilis deleniti assumenda. Molestiae provident expedita cupiditate corrupti eum eaque ea at. Occaecati perspiciatis velit voluptas alias ullam consectetur explicabo. Ullam cupiditate dolor maxime dolores quos.
Ullam quos dolores aspernatur dolorem dolores. Sint placeat at quibusdam praesentium impedit commodi optio. Id quia commodi vitae tenetur. Eligendi expedita porro voluptas sapiente.
Dolor distinctio exercitationem labore dolorum. Dolores exercitationem sequi molestias fugit aut. Error tempore aperiam adipisci. At maxime ullam pariatur dolorem excepturi.
Suscipit dol

4. Вывод результатов кодирования
----------------------------------------------------

In [4]:
encoded_text, codes_num = encode_lz78(text)
    
print '\nДлина закодированного текста в битах: {}'.format(len(encoded_text))
print 'Длина исходного текста в битах: {}'.format(len(text)*8)
print 'Сжатие: {:.2f}%'.format(100*(1 - len(encoded_text)/float(len(text)*8)))

Количество переполнений буффера 3
Количество пар 3094
Время выполнения 1.545

Длина закодированного текста в битах: 55692
Длина исходного текста в батах: 78944
Сжатие: 29.45%


In [5]:
print encoded_text[:1000] + '...'

0000000000010101100000000000011001010000000000011100100000000000011010010000000000011101000000000000011000010000000101011010010000000000011100110000000000001000000000000000011010000000000110011100100000000000011101010000000000011011010000001001011000010000000000011100000000000010011100100000000100011000010000001101001000000000001111011100100000000110011001010000001000011001010000000000011011100000000111011101010000010010011101000000000010011011010000001111011011110000000011011010010000000000011000100000001100011100110000001001011100100000000010011100000000001100011001000000010001011011100000000000011001000000010100001011100000001001010101000000000000011011110000000101011000010000010010011000010000001000011100000000010000011010010000100101011100100000000010011100110000001001011000100000000000011011000000000110011011100000100010011010010000000111011010010000001000001000000000000000011101100000100101011011000000001100011100000000100110011100110000001001011011010000110011011011000000000100

5. Функция декодирования
------------------------------------------------

In [6]:
def decode_lz78(text):
    
    dictionary = []
    
    # Для каждой пары
    decoded_text = ''
    for i in xrange(codes_num):
        code = text[i*(CODE_LENGTH): (i + 1)*CODE_LENGTH]
        element_0 = int(code[:CODE_BYTE_LENGTH], 2)
        element_1 = chr(int(code[CODE_BYTE_LENGTH:], 2))
        
        if element_0:
            decoded_text += dictionary[element_0 - 1] + element_1
            dictionary.append(dictionary[element_0 - 1] + element_1)
        else:
            decoded_text += element_1
            dictionary.append(element_1)
            
    return decoded_text  

6. Результат декодирования
----------------------------------------------------

In [7]:
print decode_lz78(encoded_text)[:1000] + '...'

Veritatis harum aperiam praesentium temporibus repudiandae. Totam asperiores blanditiis voluptas mollitia. Praesentium aut tenetur voluptates nam ducimus.
In ullam fuga at totam. Eligendi reprehenderit sapiente sapiente cum error ea. Ad dignissimos voluptate inventore cum ratione quae. Necessitatibus iste excepturi itaque pariatur. Repudiandae consectetur maiores temporibus quo autem vero illo.
Nesciunt tempora atque facilis deleniti assumenda. Molestiae provident expedita cupiditate corrupti eum eaque ea at. Occaecati perspiciatis velit voluptas alias ullam consectetur explicabo. Ullam cupiditate dolor maxime dolores quos.
Ullam quos dolores aspernatur dolorem dolores. Sint placeat at quibusdam praesentium impedit commodi optio. Id quia commodi vitae tenetur. Eligendi expedita porro voluptas sapiente.
Dolor distinctio exercitationem labore dolorum. Dolores exercitationem sequi molestias fugit aut. Error tempore aperiam adipisci. At maxime ullam pariatur dolorem excepturi.
Suscipit dol